In [2]:
from dotenv import load_dotenv
import enum
import instructor
from openai import OpenAI
import os
from pathlib import Path
from pprint import pprint as pp
from pydantic import BaseModel, Field, StringConstraints, conlist, field_validator
from typing import List, Union
from typing_extensions import Annotated

In [3]:
# load API key

dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# main constants

GPT_MODEL = "gpt-4o" # points to latest GPT model

#instantiate client
client = instructor.from_openai(OpenAI(), mode=instructor.Mode.TOOLS)

In [4]:
# query plan primitives

class QueryType(str, enum.Enum):
    """Enumeration representing the types of queries that can be asked to a question answer system."""

    SINGLE_QUESTION = "SINGLE"
    MERGE_MULTIPLE_RESPONSES = "MERGE_MULTIPLE_RESPONSES"

class Query(BaseModel):
    """Class representing a single question in a query plan."""

    id: int = Field(..., description="Unique id of the query")
    question: str = Field(
        ...,
        description="Question asked using a question answering system",
    )
    dependencies: List[int] = Field(
        default_factory=list,
        description="List of sub questions that need to be answered before asking this question",
    )
    node_type: QueryType = Field(
        default=QueryType.SINGLE_QUESTION,
        description="Type of question, either a single question or a multi-question merge",
    )

class QueryPlan(BaseModel):
    """Container class representing a tree of questions to ask a question answering system."""

    query_graph: List[Query] = Field(
        ..., description="The query graph representing the plan"
    )

    def _dependencies(self, ids: List[int]) -> List[Query]:
        """Returns the dependencies of a query given their ids."""

        return [q for q in self.query_graph if q.id in ids]

In [6]:
# revised wrapper

def wrapper(
    system_prompt: str | None = None, 
    user_prompt: Union[str, list] | None = None, 
    response_model: BaseModel | None = None, 
    max_retries: int = 3, 
    additional_messages: Union[str, List[str]] | None = None
):
    """Wrapper function to generate LLM completion"""
    messages = []

    # Add system prompt if provided
    if system_prompt is not None:
        messages.append({"role": "system", "content": system_prompt})

    # Add additional messages before user_prompt
    if additional_messages is not None:
        if isinstance(additional_messages, list):
            for message in additional_messages:
                messages.append({"role": "user", "content": message})
        else:
            messages.append({"role": "user", "content": additional_messages})

    # Add user context if provided
    if user_prompt is not None:
        if isinstance(user_prompt, list):
            for context in user_prompt:
                messages.append({"role": "user", "content": context})
        else:
            messages.append({"role": "user", "content": user_prompt})

    # Generate the completion
    completion = client.chat.completions.create(
        model=GPT_MODEL,
        response_model=response_model,
        max_retries=max_retries,
        messages=messages
    )
    return completion


In [7]:
# suggestions to optimise query plan

class SuggestionList(BaseModel):
    """A detailed list of suggestions to optimise the query plan"""

    suggestions: List[str]

In [8]:
plan_system_prompt = """
You are a world class query planning algorithm capable of breaking apart questions into its dependency queries such that the answers can be used to inform the parent question. 
Do not answer the questions, simply provide a correct compute graph with good specific questions to ask and relevant dependencies.
Your compute graph should both decompose and recompose queries in order to answer the parent question.
Before you call the function, think step-by-step to get a better understanding of the problem.
"""

optimise_system_prompt = """
You are a highly sophisticated query optimization algorithm designed to enhance the quality of query plans.
Your primary function is not to answer the queries but to meticulously improve the query graph. 
This involves ensuring each step is clearly defined and leverages detailed dependencies to construct a comprehensive and precise query sequence.
Before making any modifications, thoroughly evaluate the existing query plan to identify opportunities where additional or more detailed questions or steps could enhance clarity or add value.  
Your output should detail an optimised query graph where each question is well-defined and logically sequenced based on its dependencies.
Ensure no loss of necessary detail and increase the questions or steps where it adds to the robustness and precision of the information-gathering process.
"""

In [30]:
parent_question = "What are a detailed list of general objections to an AI platform team in a B2B SAAS company? I want precise and clearly understandable questions"

plan_user_prompt = f"""
Consider the parent question: {parent_question}
Generate the correct, complete query plan to answer the parent question.
"""

In [23]:
# initial response

response = wrapper(
    system_prompt=plan_system_prompt,
    user_prompt=plan_user_prompt,
    response_model=QueryPlan,
    max_retries=3
)

In [31]:
# response with additional messages as context 

additional_messages = """
This is a context of an AI platform team

---

Platform

Innovations are adopted quickly if they are easy to make, easy to use, fit existing patterns, and have visible results. Generative AI has abruptly changed the innovation landscape by accelerating the utility and availability of AI products and services.
While the vast majority of consumers and companies do not make technology, they are actively searching for value in this space. Consequently, technology companies are trying to implement AI features that create value for their customers as fast as possible.
The long tail of B2B SaaS companies has a unique set of challenges. Many of these companies have enterprise client traction and skilled development teams but lack extra capital and in-house AI/ML resources, making it hard to bootstrap production grade AI capabilities and features.


Goals
A dedicated platform team helps B2B SaaS companies stay competitive by quickly matching competitors' advances and meeting client expectations for new experiences.
This team drives innovation, positions the company as a leader in AI, and reduces risk by providing systemic infrastructure and resources to prevent project failures.
It specifically helps:
Product teams launch AI features
Business teams automate workflows
Leaders unify AI strategy, standards and voice
Managers make data-driven decisions
Clients adopt third-party AI products
Staff upskill AI knowledge and fluency
Ultimately, platform teams shape organisational culture, ensuring that responsibility and enthusiasm for AI are shared by everyone, not just a small group of evangelists.
Good platform teams function as skunkworks, rapid execution engines, and guardrails, ensuring the company quickly and safely integrates AI across all operations in order to benefit clients and establish AI leadership amongst competitors.


Ideas
When bootstrapping platform teams, only speed and outcomes matter. Fast, cheap value creation determines whether platforms continue or are killed off because resources are limited, track records nonexistent and executive patience is thin.
Platform teams must act for all functions and departments in a B2B SaaS company. They should tackle cross disciplinary needs - at least product, design and engineering to start with - and champion fairness - equitable platform access to all products - as foundational principles for success.
Focusing on both technical tooling and service delivery is important, as the latter reduces early bottlenecks the most.
In this environment, prioritisation is king and selecting the correct set of valuable and feasible initiatives to iterate on and build company confidence is crucial.
Platform teams empower executives with a robust reporting framework that simplifies tracking, optimises resource allocation, and sharpens decision-making regarding AI initiatives.
Goal 1 : Help product teams launch AI features


KPI
Bottleneck
Actions
Write PRD in 1 day 
Lack of competitor awareness and AI expertise slows opportunity identification
1. Searchable, updated competitor and client database of AI features

2. Custom GPT for rapid ideation and document formatting
Develop prototype in 1 week
Lack of tooling slows conversion from idea to PoC
1. Curated, ready-to-go notebooks with composable components and pipelines to bootstrap a project

2. [Optional] Buy a low/no-code service to help non-engineers prototype
Create pitch in 1 day

Approval decision in 1 hour
Lack of standard templates, explicit criteria and quick feedback slows generation and evaluation
1. AI-powered templates to auto-construct pitches and provide instant feedback for human editing

2. Automated evaluation reports and pipelines to expedite committee approval

3. Access to platform team experts for direct consultation and recommendations


Build AI feature in 6 weeks
Setting up infrastructure, experimenting, and applying advanced techniques takes too much time
1. Maintain a quick-start list of managed services infrastructure for zero friction project setup and deployment

2. Provide an out-of-the-box observability platform for automatic monitoring and evaluations

3. Abstract technical infrastructure and best practices around advanced features like RAG, Fine Tuning, Agent, Multimodal

4. Synthetic data pipelines for bootstrapping teams working with confidential data

5. Dedicated platform resources (e.g. engineers, designer) pairing with product teams to accelerate delivery
Compliance check in 1 day
Staying updated with regulatory frameworks, assessing their impact on product features, and preparing external documents is time-consuming.
1. Maintain a searchable, updated compliance directory to track and update local, state, national, and international regulatory frameworks.

2. Use RAG-based rubric evaluation for feature compliance

3. Automated drafting of approval request letters to appropriate stakeholders
GTM in 1 week




Post-Launch Maintenance for 3 months
Maintaining AI features post-launch is resource-intensive, requiring continuous monitoring and a different approach for generative AI's probabilistic nature
1. Dedicated hypercare by AI platform team to handle bugs, fixes and updates for 3 months post launch

2. Set up automated monitoring systems to track AI performance and detect issues early.



We need a wartime platform squad to stay ahead. Wartime squads move fast, cut red tape, and prioritise impact. They focus on speed and value, driving the company forward by launching AI features quickly. Leaders make tough calls to keep projects on track.
This team works seamlessly with product and business teams, enabling them and ruthlessly prioritising the best ideas. They eliminate bureaucracy, integrate new AI tools swiftly, and handle issues head-on. They ensure compliance and safety standards are met without slowing down progress.
Unlike peacetime squads that maintain stability, wartime squads thrive in urgency. They deliver immediate results and keep the company ahead of competitors. Quick decisions and a relentless focus on outcomes make them essential for AI leadership and rapid innovation.




Buy or build




"""
response = wrapper(
    system_prompt=plan_system_prompt,
    user_prompt=plan_user_prompt,
    response_model=QueryPlan,
    max_retries=3,
    additional_messages=additional_messages
)


In [32]:
for query in response.query_graph:
    print(query)


id=1 question='What are the general objections to an AI platform team in a B2B SAAS company?' dependencies=[2, 3, 4] node_type=<QueryType.SINGLE_QUESTION: 'SINGLE'>
id=2 question='What objections do product teams typically have towards an AI platform team?' dependencies=[5, 6, 7, 8, 9, 10] node_type=<QueryType.SINGLE_QUESTION: 'SINGLE'>
id=3 question='What objections do business teams typically have towards an AI platform team?' dependencies=[11, 12, 13, 14, 15] node_type=<QueryType.SINGLE_QUESTION: 'SINGLE'>
id=4 question='What objections do leadership teams typically have towards an AI platform team?' dependencies=[16, 17, 18, 19, 20] node_type=<QueryType.SINGLE_QUESTION: 'SINGLE'>
id=5 question='What issues do product teams encounter when creating product requirement documents with the AI platform team?' dependencies=[] node_type=<QueryType.SINGLE_QUESTION: 'SINGLE'>
id=6 question='What issues do product teams encounter when developing prototypes with the AI platform team?' dependen